In [2]:
import psycopg
import json
import pickle
import pandas as pd
import numpy as np
from tabulate import tabulate

In [3]:
conn = psycopg.connect(
    host="localhost",
    dbname="yelp_dataset",      
    user="anoutsala",         
    password=""  
)

cur = conn.cursor()

cur.execute("""
    CREATE TABLE IF NOT EXISTS businesses (
        business_id VARCHAR(22) PRIMARY KEY,
        name TEXT,
        address TEXT,
        city TEXT,
        state TEXT,
        postal_code TEXT,
        latitude FLOAT,
        longitude FLOAT,
        stars FLOAT,
        review_count INT
    );
""")

file_path = "yelp_dataset/yelp_academic_dataset_business.json"

with open(file_path, "r") as f:
    data = [json.loads(line) for line in f]

insert_query = """
    INSERT INTO businesses (
        business_id, name, address, city, state, postal_code, latitude, longitude, stars, review_count
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    ON CONFLICT (business_id) DO NOTHING;
"""

for business in data:
    cur.execute(
        insert_query,
        (
            business["business_id"],
            business.get("name"),
            business.get("address"),
            business.get("city"),
            business.get("state"),
            business.get("postal_code"),
            business.get("latitude"),
            business.get("longitude"),
            business.get("stars"),
            business.get("review_count")
        )
    )

conn.commit()
cur.close()
conn.close()

print("Businesses table created successfully!")

Businesses table created successfully!


In [4]:
conn = psycopg.connect(
    host="localhost",
    dbname="yelp_dataset",
    user="anoutsala",
    password=""
)

cur = conn.cursor()

cur.execute("""
    CREATE TABLE IF NOT EXISTS users (
        user_id VARCHAR(22) PRIMARY KEY,            
        name TEXT,                           
        review_count INT,                    
        yelping_since DATE,                  
        useful INT,                          
        funny INT,                           
        cool INT,                            
        fans INT,                            
        average_stars FLOAT,                 
        compliment_hot INT,                 
        compliment_more INT,                 
        compliment_profile INT,              
        compliment_cute INT,                 
        compliment_list INT,                 
        compliment_note INT,                 
        compliment_plain INT,                
        compliment_cool INT,                 
        compliment_funny INT,                
        compliment_writer INT,               
        compliment_photos INT                
    );
""")

conn.commit()
cur.close()
conn.close()

In [4]:
conn = psycopg.connect(
    host="localhost",
    dbname="yelp_dataset",      
    user="anoutsala",         
    password=""  
)

cur = conn.cursor()

file_path_users = "yelp_dataset/yelp_academic_dataset_user.json"

with open(file_path_users, "r") as f:
    users_data = [json.loads(line) for line in f]

insert_users_query = """
    INSERT INTO users (
        user_id, name, review_count, yelping_since, useful, funny, cool, fans,
        average_stars, compliment_hot, compliment_more, compliment_profile,
        compliment_cute, compliment_list, compliment_note, compliment_plain,
        compliment_cool, compliment_funny, compliment_writer, compliment_photos
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    ON CONFLICT (user_id) DO NOTHING;
"""

for user in users_data:
    cur.execute(
        insert_users_query,
        (
            user["user_id"],
            user.get("name"),
            user.get("review_count"),
            user.get("yelping_since"),
            user.get("useful"),
            user.get("funny"),
            user.get("cool"),
            user.get("fans"),
            user.get("average_stars"),
            user.get("compliment_hot"),
            user.get("compliment_more"),
            user.get("compliment_profile"),
            user.get("compliment_cute"),
            user.get("compliment_list"),
            user.get("compliment_note"),
            user.get("compliment_plain"),
            user.get("compliment_cool"),
            user.get("compliment_funny"),
            user.get("compliment_writer"),
            user.get("compliment_photos")
        )
    )

conn.commit()
cur.close()
conn.close()

print("Users table created successfully!")

Connected to PostgreSQL!


In [36]:
conn = psycopg.connect(
    host="localhost",
    dbname="yelp_dataset",
    user="anoutsala",
    password=""
)

cur = conn.cursor()

cur.execute("""
    CREATE TABLE IF NOT EXISTS reviews (
        review_id VARCHAR(22) PRIMARY KEY,
        user_id VARCHAR(22),
        business_id VARCHAR(22),
        stars INT,
        date DATE,
        text TEXT,
        useful INT,
        funny INT,
        cool INT,
        CONSTRAINT fk_user FOREIGN KEY (user_id) REFERENCES users(user_id),
        CONSTRAINT fk_business FOREIGN KEY (business_id) REFERENCES businesses(business_id)
    );
""")

cur.execute("SELECT user_id FROM users;")
valid_user_ids = {row[0] for row in cur.fetchall()} 

file_path_reviews = "yelp_dataset/yelp_academic_dataset_review.json"

with open(file_path_reviews, "r") as f:
    reviews_data = [json.loads(line) for line in f]

filtered_reviews = [review for review in reviews_data if review["user_id"] in valid_user_ids]

insert_reviews_query = """
    INSERT INTO reviews (
        review_id, user_id, business_id, stars, date, text, useful, funny, cool
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    ON CONFLICT (review_id) DO NOTHING;
"""

for review in filtered_reviews:
    cur.execute(
        insert_reviews_query,
        (
            review["review_id"],
            review["user_id"],
            review["business_id"],
            review.get("stars"), 
            review.get("date"),
            review.get("text"),
            review.get("useful"),
            review.get("funny"),
            review.get("cool")
        )
    )

conn.commit()
cur.close()
conn.close()

print("Reviews table created successfully!")

Connected to PostgreSQL!


In [12]:
conn = psycopg.connect(
    host="localhost",
    dbname="yelp_dataset",
    user="anoutsala",
    password=""
)

cur = conn.cursor()

# valid users ID
cur.execute("SELECT user_id FROM users;")
valid_user_ids = {row[0] for row in cur.fetchall()}

file_path_reviews = "yelp_dataset/yelp_academic_dataset_review.json"

with open(file_path_reviews, "r") as f:
    reviews_data = [json.loads(line) for line in f]

# find the user_id values not in the users table
invalid_user_ids = {review["user_id"] for review in reviews_data if review["user_id"] not in valid_user_ids}

cur.execute("""
    CREATE TABLE IF NOT EXISTS not_in_users (
        user_id VARCHAR(22) PRIMARY KEY
    );
""")

insert_query = "INSERT INTO not_in_users (user_id) VALUES (%s) ON CONFLICT (user_id) DO NOTHING;"
for user_id in invalid_user_ids:
    cur.execute(insert_query, (user_id,))

conn.commit()
print(f"Inserted {len(invalid_user_ids)} user_id values into the not_in_users table.")
cur.close()
conn.close()



Connected to PostgreSQL!
Inserted 32 user_id values into the not_in_users table.


In [2]:
conn = psycopg.connect(
    host="localhost",
    dbname="yelp_dataset",
    user="anoutsala",
    password=""
)

cur = conn.cursor()

cur.execute("SELECT count(*) FROM not_in_users;")

results = cur.fetchall()
for row in results:
    print(row)

cur.close()
conn.close()


Connected to PostgreSQL!
(32,)


In [3]:
conn = psycopg.connect(
    host="localhost",
    dbname="yelp_dataset",
    user="anoutsala",
    password=""
)

cur = conn.cursor()
print("Connected to PostgreSQL!")

cur.execute("SELECT COUNT(DISTINCT user_id) FROM not_in_users;")

results = cur.fetchall()
for row in results:
    print(row)

cur.close()
conn.close()

Connected to PostgreSQL!
(32,)


In [13]:
conn = psycopg.connect(
    host="localhost",
    dbname="yelp_dataset",
    user="anoutsala",
    password=""
)

cur = conn.cursor()
print("Connected to PostgreSQL!")

cur.execute("""
    CREATE TABLE IF NOT EXISTS unfiltered_reviews (
        review_id VARCHAR(22) PRIMARY KEY,
        user_id VARCHAR(22),
        business_id VARCHAR(22),
        stars INT,
        date DATE,
        text TEXT,
        useful INT,
        funny INT,
        cool INT
    );
""")

file_path_reviews = "yelp_dataset/yelp_academic_dataset_review.json"

with open(file_path_reviews, "r") as f:
    reviews_data = [json.loads(line) for line in f]

insert_reviews_query = """
    INSERT INTO unfiltered_reviews (
        review_id, user_id, business_id, stars, date, text, useful, funny, cool
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    ON CONFLICT (review_id) DO NOTHING;
"""

for review in reviews_data:
    cur.execute(
        insert_reviews_query,
        (
            review["review_id"],
            review["user_id"],
            review["business_id"],
            review.get("stars"),
            review.get("date"),
            review.get("text"),
            review.get("useful"),
            review.get("funny"),
            review.get("cool")
        )
    )

conn.commit()
cur.close()
conn.close()
print("Created unfiltered_reviews table.")

Connected to PostgreSQL!
Unfiltered review data inserted into 'unfiltered_reviews'.


In [14]:
conn = psycopg.connect(
    host="localhost",
    dbname="yelp_dataset",
    user="anoutsala",
    password=""
)

cur = conn.cursor()

cur.execute("""
    SELECT COUNT(*)
    FROM not_in_users u
    JOIN unfiltered_reviews r ON u.user_id = r.user_id
    JOIN businesses_in_tampa b ON r.business_id = b.business_id;
""")

count = cur.fetchone()[0]
if count > 0:
    print(f"There are {count} user_id values from 'not_in_users' who made reviews on 'businesses_in_tampa'.")
else:
    print("No user_id from 'not_in_users' made a review on 'businesses_in_tampa'.")

cur.close()
conn.close()


There are 3 user_id values from 'not_in_users' who made reviews on 'businesses_in_tampa'.


In [2]:
conn = psycopg.connect(
    host="localhost",
    dbname="yelp_dataset",
    user="anoutsala",
    password=""
)

cur = conn.cursor()

cur.execute("""
    SELECT r.review_id, r.user_id, b.name
    FROM not_in_users u
    JOIN unfiltered_reviews r ON u.user_id = r.user_id
    JOIN businesses_in_tampa b ON r.business_id = b.business_id;
""")

invalid_users_in_tp_reviews = cur.fetchall()

columns = [desc[0] for desc in cur.description]

print(tabulate(invalid_users_in_tp_reviews, headers=columns, tablefmt="psql"))

cur.close()
conn.close()


+------------------------+------------------------+-----------------------------------+
| review_id              | user_id                | name                              |
|------------------------+------------------------+-----------------------------------|
| ebQaTudrnT1wE8SZkzzYGQ | 5XiPz5mJK_RtJQVkXIqxYg | Eat Right                         |
| G8n21nKcVNR-noZJqz3sEw | vq2H7lJ73VwXMDqC8DiImw | Florida Building Inspection Group |
| wIW-gvYpTWsLMRix-8oPBQ | NCeW1I6C4K7qhY4kRH8cOA | 717 Parking Enterprises           |
+------------------------+------------------------+-----------------------------------+


In [5]:
conn = psycopg.connect(
    host="localhost",
    dbname="yelp_dataset",
    user="anoutsala",
    password=""
)

cur = conn.cursor()

cur.execute("""
    CREATE OR REPLACE VIEW businesses_in_fl AS
    SELECT *
    FROM businesses
    WHERE state = 'FL';""")

cur.execute("""
    CREATE OR REPLACE VIEW businesses_in_tampa AS
    SELECT *
    FROM businesses_in_fl
    WHERE city = 'Tampa'
    OR city = 'Tampa Bay'
    OR city = 'tampa'
    OR city = 'TAMPA';""")

cur.execute("""
    CREATE OR REPLACE VIEW tampa_business_reviews AS
    SELECT r.*
    FROM businesses_in_tampa b JOIN reviews r
    ON b.business_id = r.business_id;""")


cur.execute("""
    CREATE OR REPLACE VIEW tampa_reviews AS
    SELECT r.*
    FROM reviews r 
    JOIN (
        SELECT DISTINCT business_id
        FROM businesses_in_tampa) AS b
    ON b.business_id = r.business_id;""")

cur.close()
conn.commit()
conn.close()

In [6]:
conn = psycopg.connect(
    host="localhost",
    dbname="yelp_dataset",
    user="anoutsala",
    password=""
)

cur = conn.cursor()

cur.execute("""
    SELECT city, count(*)
    FROM businesses_in_fl
    GROUP BY city
    ORDER BY count DESC
    LIMIT 20;
""")

count_in_fl = cur.fetchall()

columns = [desc[0] for desc in cur.description]

print(tabulate(count_in_fl, headers=columns, tablefmt="psql"))

cur.close()
conn.close()

+------------------+---------+
| city             |   count |
|------------------+---------|
| Tampa            |    9048 |
| Clearwater       |    2221 |
| Saint Petersburg |    1663 |
| St. Petersburg   |    1185 |
| Brandon          |    1033 |
| Largo            |    1002 |
| Palm Harbor      |     665 |
| New Port Richey  |     604 |
| Lutz             |     591 |
| Riverview        |     588 |
| Wesley Chapel    |     560 |
| Pinellas Park    |     512 |
| Dunedin          |     490 |
| Spring Hill      |     400 |
| Tarpon Springs   |     398 |
| St Petersburg    |     387 |
| Seminole         |     359 |
| Oldsmar          |     286 |
| Valrico          |     276 |
| Port Richey      |     270 |
+------------------+---------+


In [7]:
conn = psycopg.connect(
    host="localhost",
    dbname="yelp_dataset",
    user="anoutsala",
    password=""
)

cur = conn.cursor()

cur.execute("""
    DROP VIEW IF EXISTS businesses_in_top3;
""")

cur.execute("""
    CREATE VIEW businesses_in_top3 AS
    SELECT city, stars
    FROM businesses_in_fl
    WHERE city IN ('Tampa', 'Clearwater', 'St Petersburg', 'St. Petersburg', 'Saint Petersburg');
""")

conn.commit()
cur.close()
conn.close()

In [8]:
conn = psycopg.connect(
    host="localhost",
    dbname="yelp_dataset",
    user="anoutsala",
    password=""
)

cur = conn.cursor()

cur.execute("""
    SELECT 
    city_grouped AS city,
    AVG(stars) AS avg_review_score
    FROM (
        SELECT 
            CASE
                WHEN city IN ('St Petersburg', 'St. Petersburg', 'Saint Petersburg') THEN 'Saint Petersburg'
                ELSE city
            END AS city_grouped,
            stars
        FROM businesses_in_top3
    ) AS grouped_data
    GROUP BY city_grouped
    ORDER BY avg_review_score DESC;
""")

count_in_fl = cur.fetchall()

columns = [desc[0] for desc in cur.description]

print(tabulate(count_in_fl, headers=columns, tablefmt="psql"))

cur.close()
conn.close()

+------------------+--------------------+
| city             |   avg_review_score |
|------------------+--------------------|
| Saint Petersburg |            3.74297 |
| Clearwater       |            3.60131 |
| Tampa            |            3.58306 |
+------------------+--------------------+


In [3]:
conn = psycopg.connect(
    host="localhost",
    dbname="yelp_dataset",
    user="anoutsala",
    password=""
)

cur = conn.cursor()

cur.execute("""
    EXPLAIN ANALYZE
    SELECT 
    city_grouped AS city,
    AVG(stars) AS avg_review_score
    FROM (
        SELECT 
            CASE
                WHEN city IN ('St Petersburg', 'St. Petersburg', 'Saint Petersburg') THEN 'Saint Petersburg'
                ELSE city
            END AS city_grouped,
            stars
        FROM businesses_in_top3
    ) AS grouped_data
    GROUP BY city_grouped
    ORDER BY avg_review_score DESC;
""")

execution_plan_top3 = cur.fetchall()
for line in execution_plan_top3:
    print(line[0])

cur.close()

Sort  (cost=5547.65..5549.59 rows=776 width=40) (actual time=268.209..271.910 rows=3 loops=1)
  Sort Key: (avg(businesses.stars)) DESC
  Sort Method: quicksort  Memory: 25kB
  ->  Finalize GroupAggregate  (cost=5404.67..5510.40 rows=776 width=40) (actual time=268.134..271.839 rows=3 loops=1)
        Group Key: (CASE WHEN (businesses.city = ANY ('{"St Petersburg","St. Petersburg","Saint Petersburg"}'::text[])) THEN 'Saint Petersburg'::text ELSE businesses.city END)
        ->  Gather Merge  (cost=5404.67..5493.91 rows=776 width=64) (actual time=268.120..271.823 rows=6 loops=1)
              Workers Planned: 1
              Workers Launched: 1
              ->  Sort  (cost=4404.66..4406.60 rows=776 width=64) (actual time=250.662..250.664 rows=3 loops=2)
                    Sort Key: (CASE WHEN (businesses.city = ANY ('{"St Petersburg","St. Petersburg","Saint Petersburg"}'::text[])) THEN 'Saint Petersburg'::text ELSE businesses.city END)
                    Sort Method: quicksort  Memory:

In [9]:
conn = psycopg.connect(
    host="localhost",
    dbname="yelp_dataset",
    user="anoutsala",
    password=""
)

cur = conn.cursor()

cur.execute("""
    SELECT 
        r.user_id, 
        u.name AS user_name, 
        COUNT(*) AS review_count, 
        AVG(r.stars) AS avg_stars 
    FROM tampa_reviews r
    JOIN users u
    ON r.user_id = u.user_id
    GROUP BY r.user_id, u.name
    HAVING COUNT(*) > 5
    ORDER BY avg_stars ASC, review_count DESC, user_name ASC
    LIMIT 10;
""")

hater = cur.fetchall()

columns = [desc[0] for desc in cur.description]

print(tabulate(hater, headers=columns, tablefmt="psql"))

cur.close()
conn.close()

+------------------------+-------------+----------------+-------------+
| user_id                | user_name   |   review_count |   avg_stars |
|------------------------+-------------+----------------+-------------|
| C7fbmhCmXXhtWRZyW6Nzmg | A           |             14 |           1 |
| k2lXKvz4iSw0OZSSojCkdw | Bob-A-Lou   |             10 |           1 |
| YgeZ-tJ0ZEU_m41pGBng_A | Carlos      |             10 |           1 |
| CGp3KducoS3lemPVanz0Xw | Lola        |              9 |           1 |
| mvUw8RPzmXkVdCR4Pr_WJg | Mauro       |              9 |           1 |
| OL87RR63x07Y0K2xpCG4Aw | Nicole      |              9 |           1 |
| fluPpH9ap58Y5xkPPyFQuQ | Rick        |              9 |           1 |
| EXL55TgQAo1RRUIpZvKmWQ | Sandy       |              9 |           1 |
| mAI_E0rVjyC4FF9PSn37rA | Terry       |              9 |           1 |
| zHKbREByeKvgbKPxbKIweA | Vic         |              9 |           1 |
+------------------------+-------------+----------------+-------

In [10]:
conn = psycopg.connect(
    host="localhost",
    dbname="yelp_dataset",
    user="anoutsala",
    password=""
)

cur = conn.cursor()

cur.execute("""
    EXPLAIN ANALYZE
    SELECT 
        r.user_id, 
        u.name AS user_name, 
        COUNT(*) AS review_count, 
        AVG(r.stars) AS avg_stars 
    FROM tampa_reviews r
    JOIN users u
    ON r.user_id = u.user_id
    GROUP BY r.user_id, u.name
    HAVING COUNT(*) > 5
    ORDER BY avg_stars ASC, review_count DESC, user_name ASC
    LIMIT 10;
""")

execution_plan_hater = cur.fetchall()
for line in execution_plan_hater:
    print(line[0])

cur.close()
conn.close()

Limit  (cost=730180.22..730180.25 rows=10 width=69) (actual time=22906.210..22968.152 rows=10 loops=1)
  ->  Sort  (cost=730180.22..730403.89 rows=89466 width=69) (actual time=22906.202..22968.143 rows=10 loops=1)
        Sort Key: (avg(r.stars)), (count(*)) DESC, u.name
        Sort Method: top-N heapsort  Memory: 26kB
        ->  Finalize GroupAggregate  (cost=693651.40..728246.89 rows=89466 width=69) (actual time=22654.609..22963.643 rows=12987 loops=1)
              Group Key: r.user_id, u.name
              Filter: (count(*) > 5)
              Rows Removed by Filter: 153954
              ->  Gather Merge  (cost=693651.40..721984.25 rows=223666 width=69) (actual time=22654.422..22840.895 rows=167735 loops=1)
                    Workers Planned: 2
                    Workers Launched: 2
                    ->  Partial GroupAggregate  (cost=692651.38..695167.62 rows=111833 width=69) (actual time=22636.556..22719.974 rows=55912 loops=3)
                          Group Key: r.user_id, 

In [11]:
conn = psycopg.connect(
    host="localhost",
    dbname="yelp_dataset",
    user="anoutsala",
    password=""
)

cur = conn.cursor()

cur.execute("""
    WITH single_reviews AS (
        SELECT r.business_id, r.user_id, COUNT(*) AS review_count, AVG(r.stars) AS avg_stars
        FROM tampa_reviews r
        GROUP BY r.business_id, r.user_id
        HAVING COUNT(*) = 1
    ),
    business_single_review_counts AS (
        SELECT business_id,                                     
               COUNT(*) AS single_review_user_count, 
               ROUND(AVG(avg_stars), 2) AS avg_stars_for_single_reviews
        FROM single_reviews
        GROUP BY business_id
    )                                   
    SELECT b.business_id, 
           tb.name AS business_name, 
           single_review_user_count, 
           avg_stars_for_single_reviews                 
    FROM business_single_review_counts b
    JOIN businesses_in_tampa tb
    ON b.business_id = tb.business_id
    ORDER BY single_review_user_count DESC 
    LIMIT 3;
""")

single_reviews = cur.fetchall()

columns = [desc[0] for desc in cur.description]

print(tabulate(single_reviews, headers=columns, tablefmt="psql"))

cur.close()
conn.close()

+------------------------+--------------------+----------------------------+--------------------------------+
| business_id            | business_name      |   single_review_user_count |   avg_stars_for_single_reviews |
|------------------------+--------------------+----------------------------+--------------------------------|
| QHWYlmVbLC3K6eglWoHVvA | Datz               |                       3148 |                           4.14 |
| L5LLN0RafiV1Z9cddzvuCw | Ulele              |                       2962 |                           4.17 |
| dsfRniRgfbDjC8os848B6A | Bern's Steak House |                       2831 |                           4.26 |
+------------------------+--------------------+----------------------------+--------------------------------+


In [12]:
conn = psycopg.connect(
    host="localhost",
    dbname="yelp_dataset",
    user="anoutsala",
    password=""
)

cur = conn.cursor()

cur.execute("""
    EXPLAIN ANALYZE
    WITH single_reviews AS (
        SELECT r.business_id, r.user_id, COUNT(*) AS review_count, AVG(r.stars) AS avg_stars
        FROM tampa_reviews r
        GROUP BY r.business_id, r.user_id
        HAVING COUNT(*) = 1
    ),
    business_single_review_counts AS (
        SELECT business_id,                                     
               COUNT(*) AS single_review_user_count, 
               ROUND(AVG(avg_stars), 2) AS avg_stars_for_single_reviews
        FROM single_reviews
        GROUP BY business_id
    )                                   
    SELECT b.business_id, 
           tb.name AS business_name, 
           single_review_user_count, 
           avg_stars_for_single_reviews                 
    FROM business_single_review_counts b
    JOIN businesses_in_tampa tb
    ON b.business_id = tb.business_id
    ORDER BY single_review_user_count DESC 
    LIMIT 3;
""")

execution_plan_single_reviews = cur.fetchall()
for line in execution_plan_single_reviews:
    print(line[0])

cur.close()
conn.close()

Limit  (cost=664542.92..664542.93 rows=2 width=83) (actual time=19295.118..19310.561 rows=3 loops=1)
  ->  Sort  (cost=664542.92..664542.93 rows=2 width=83) (actual time=19295.117..19310.559 rows=3 loops=1)
        Sort Key: (count(*)) DESC
        Sort Method: top-N heapsort  Memory: 25kB
        ->  Nested Loop  (cost=628349.34..664542.91 rows=2 width=83) (actual time=18336.522..19306.118 rows=9207 loops=1)
              ->  GroupAggregate  (cost=628348.92..662970.90 rows=200 width=63) (actual time=18336.425..19237.470 rows=9207 loops=1)
                    Group Key: r.business_id
                    ->  Finalize GroupAggregate  (cost=628348.92..662944.42 rows=1342 width=86) (actual time=18336.353..19147.470 rows=429298 loops=1)
                          Group Key: r.business_id, r.user_id
                          Filter: (count(*) = 1)
                          Rows Removed by Filter: 13667
                          ->  Gather Merge  (cost=628348.92..656681.77 rows=223666 width=86

In [26]:
conn = psycopg.connect(
    host="localhost",
    dbname="yelp_dataset",
    user="anoutsala",
    password=""
)

cur = conn.cursor()

cur.execute("""
    SELECT 
        postal_code, 
        COUNT(*) AS total_businesses
    FROM businesses_in_tampa
    GROUP BY postal_code ORDER BY COUNT(*) DESC LIMIT 10;
""")

postal = cur.fetchall()

columns = [desc[0] for desc in cur.description]

print(tabulate(postal, headers=columns, tablefmt="psql"))

cur.close()
conn.close()

+---------------+--------------------+
|   postal_code |   total_businesses |
|---------------+--------------------|
|         33607 |                794 |
|         33602 |                649 |
|         33609 |                639 |
|         33618 |                591 |
|         33612 |                580 |
|         33606 |                509 |
|         33629 |                496 |
|         33614 |                434 |
|         33611 |                413 |
|         33647 |                410 |
+---------------+--------------------+


In [21]:
conn = psycopg.connect(
    host="localhost",
    dbname="yelp_dataset",
    user="anoutsala",
    password=""
)

cur = conn.cursor()

cur.execute("""
    SELECT 
    COUNT(DISTINCT postal_code) AS distinct_postal_codes
    FROM businesses_in_tampa;
""")

postal = cur.fetchall()

columns = [desc[0] for desc in cur.description]

print(tabulate(postal, headers=columns, tablefmt="psql"))

cur.close()
conn.close()

+-------------------------+
|   distinct_postal_codes |
|-------------------------|
|                      97 |
+-------------------------+


In [4]:
conn = psycopg.connect(
    host="localhost",
    dbname="yelp_dataset",
    user="anoutsala",
    password=""
)

cur = conn.cursor()

cur.execute("""
    SELECT 
    MIN(total_businesses) AS min_businesses,
    MAX(total_businesses) AS max_businesses,
    AVG(total_businesses) AS avg_businesses,
    STDDEV(total_businesses) AS total_businesses
    FROM (
        SELECT postal_code, COUNT(*) AS total_businesses
        FROM businesses_in_tampa
        GROUP BY postal_code 
    ) subquery;
""")

postal_stats = cur.fetchall()

columns = [desc[0] for desc in cur.description]

print(tabulate(postal_stats, headers=columns, tablefmt="psql"))

cur.close()
conn.close()

+------------------+------------------+------------------+--------------------+
|   min_businesses |   max_businesses |   avg_businesses |   total_businesses |
|------------------+------------------+------------------+--------------------|
|                1 |              794 |          94.9175 |            184.141 |
+------------------+------------------+------------------+--------------------+
